In [ ]:
import torch
from torch import nn
from torch import optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import math
import numpy as np

import matplotlib.pyplot as plt
import tqdm
from IPython.display import clear_output

In [ ]:
class PINN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super().__init__()

        self.activation = nn.Tanh()

        self.lambda1 = nn.Parameter(torch.Tensor([math.log(1.1)]), requires_grad=True)
        self.lambda2 = nn.Parameter(torch.Tensor([math.log(0.01)]), requires_grad=True)

        self.loss_function = nn.MSELoss()

        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, *input_data):
        return self.network(torch.stack(input_data, dim=1)).flatten()


last_u_error = 0
last_pde_error = 0


def pde_loss(model, t, x, u):
    global last_u_error, last_pde_error

    t.requires_grad = True
    x.requires_grad = True

    u_pred = model(t, x)

    u_t = torch.autograd.grad(
        u_pred, t,
        grad_outputs=torch.ones_like(u_pred),
        retain_graph=True,
        create_graph=True
    )[0]
    u_x = torch.autograd.grad(
        u_pred, x,
        grad_outputs=torch.ones_like(u_pred),
        retain_graph=True,
        create_graph=True
    )[0]
    u_xx = torch.autograd.grad(
        u_x, x,
        grad_outputs=torch.ones_like(u_x),
        retain_graph=True,
        create_graph=True
    )[0]

    residue = u_t + torch.exp(model.lambda1) * u_pred * u_x - torch.exp(model.lambda2) * u_xx
    last_pde_error = residue.pow(2).mean()
    last_u_error = (u_pred - u).pow(2).mean()
    return last_u_error + last_pde_error

In [ ]:
EPOCHS = 10000
LEARNING_RATE = 0.015
SCHEDULER_RATE = 20
GAMMA = 0.9999
BATCH_SIZE = 1024

TRAINING_FRACTION = 0.95

### Dataset generation

In [ ]:
import scipy


class CustomDataset(Dataset):
    def __init__(self, t, x, u):
        super(CustomDataset).__init__()
        self.t = t
        self.x = x
        self.u = u

    def __len__(self):
        return len(self.t)

    def __getitem__(self, item):
        return self.t[item], self.x[item], self.u[item]


def load_data(path):
    data = scipy.io.loadmat(path)

    T = data['t'].flatten()[:, None]
    X = data['x'].flatten()[:, None]
    U = np.real(data['usol']).T

    t, x, u = [], [], []
    for i, t_curr in enumerate(T):
        for j, x_curr in enumerate(X):
            t.append(torch.Tensor(np.array(t_curr)))
            x.append(torch.Tensor(np.array(x_curr)))
            u.append(torch.Tensor(np.array(U[i][j])))

    t = torch.Tensor(t)
    x = torch.Tensor(x)
    u = torch.Tensor(u)

    return CustomDataset(t, x, u)


dataset = load_data("burgers_shock.mat")

training_dataset, validation_dataset = random_split(dataset,
                                                    (int(len(dataset) * TRAINING_FRACTION),
                                                     len(dataset) - int(len(dataset) * TRAINING_FRACTION)),
                                                    generator=torch.Generator().manual_seed(238)
                                                    )

training_loader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

### Network training

In [ ]:
def train_batch(model, t, x, u, optimizer):
    loss = pde_loss(model, t, x, u)
    optimizer.zero_grad()
    # maybe retain_graph=False
    loss.backward()
    optimizer.step()

    return loss.item()


def validate_batch(model, t, x, u):
    loss = pde_loss(model, t, x, u)

    return loss.item()


In [ ]:
from IPython.display import clear_output


def train(model, training_loader, validation_loader, optimizer, scheduler):
    training_loss_history, validation_loss_history = [], []

    for epoch in range(EPOCHS):
        training_loss = 0
        validation_loss = 0
        training_cnt = 0
        validation_cnt = 0

        for i, (t, x, u) in enumerate(training_loader):
            training_loss += train_batch(model, t, x, u, optimizer)
            training_cnt += 1

        for i, (t, x, u) in enumerate(validation_loader):
            validation_loss += validate_batch(model, t, x, u)
            validation_cnt += 1

        training_loss_history.append(training_loss / training_cnt)
        validation_loss_history.append(validation_loss / validation_cnt)

        if epoch % 1 == 0:
            print(f"Epoch {epoch}, PDE error: {last_pde_error.item()}, U Error: {last_u_error.item()}")
            print(
                f"Epoch {epoch}, lambda1: {torch.exp(model.lambda1).item()}, lambda2: {torch.exp(model.lambda2).item()}")
            # plot_t()

        if epoch % SCHEDULER_RATE == 0:
            scheduler.step()


LBFGS_epoch = 0


def train_LBFGS(model, dataset: CustomDataset):
    optim = torch.optim.LBFGS(
        model.parameters(),
        lr=1.0,
        max_iter=50000,
        max_eval=50000,
        history_size=50,
        tolerance_grad=1e-5,
        tolerance_change=1.0 * np.finfo(float).eps,
        line_search_fn="strong_wolfe"
    )
    t = dataset.t
    x = dataset.x
    u = dataset.u

    def closure():
        global LBFGS_epoch
        LBFGS_epoch += 1
        optim.zero_grad()
        u_pred = model(t, x)
        loss = pde_loss(model, t, x, u)
        loss.backward()
        print(f"Epoch {LBFGS_epoch}, PDE error: {last_pde_error.item()}, U Error: {last_u_error.item()}")
        print(
            f"Epoch {LBFGS_epoch}, lambda1: {torch.exp(model.lambda1).item()}, lambda2: {torch.exp(model.lambda2).item()}")
        return loss

    optim.step(closure)


In [ ]:
model = PINN(2, 1, 20)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.005)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train(model, training_loader, validation_loader, optimizer, scheduler)

In [ ]:
train_LBFGS(model, dataset)